In [14]:
# m8.predictions.read()
m6.data_1.read()

date
2016-01-04    [002319.SZA, 600643.SHA, 600058.SHA, 002741.SZ...
2016-01-05    [000863.SZA, 002285.SZA, 300094.SZA, 300189.SZ...
2016-01-06    [000502.SZA, 600387.SHA, 002326.SZA, 002592.SZ...
2016-01-07    [002625.SZA, 600097.SHA, 002146.SZA, 002605.SZ...
2016-01-08    [002169.SZA, 002074.SZA, 300166.SZA, 002416.SZ...
                                    ...                        
2016-12-26    [300507.SZA, 300501.SZA, 600379.SHA, 600623.SH...
2016-12-27    [000503.SZA, 600259.SHA, 300501.SZA, 300507.SZ...
2016-12-28    [600259.SHA, 000503.SZA, 002787.SZA, 002288.SZ...
2016-12-29    [300503.SZA, 000503.SZA, 002730.SZA, 300505.SZ...
2016-12-30    [000503.SZA, 300242.SZA, 600599.SHA, 603398.SH...
Length: 244, dtype: object

In [2]:
# 本代码由可视化策略环境自动生成 2021年12月14日 08:24
# 本代码单元只能在可视化模式下编辑。您也可以拷贝代码，粘贴到新建的代码单元或者策略，然后修改。


# Python 代码入口函数，input_1/2/3 对应三个输入端，data_1/2/3 对应三个输出端
def m6_run_bigquant_run(input_1, input_2, input_3):
    # 示例代码如下。在这里编写您的代码
    df = input_1.read()
    start = input_2.read()['start_date']
    final = df[df.date>=start].groupby('date').apply(lambda x:list(x.instrument.values[:5]))
    data_1 = DataSource().write_pickle(final)
    return Outputs(data_1=data_1)

# 后处理函数，可选。输入是主函数的输出，可以在这里对数据做处理，或者返回更友好的outputs数据格式。此函数输出不会被缓存。
def m6_post_run_bigquant_run(outputs):
    return outputs

# 回测引擎：初始化函数，只执行一次
def m19_initialize_bigquant_run(context):
    # 加载仓位数据
    context.ratios_dict = context.options['data'].read_pickle()
    
    # 系统已经设置了默认的交易手续费和滑点，要修改手续费可使用如下函数
    context.set_commission(PerOrder(buy_cost=0.0003, sell_cost=0.0013, min_cost=5))

    context.couter =0
# 回测引擎：每日数据处理函数，每天执行一次
def m19_handle_data_bigquant_run(context, data):
    if context.couter % 5 !=0:
        context.couter += 1
        return
    
    context.couter += 1
    # 按日期过滤得到今日的预测数据
    try:
        ratio_today = context.ratios_dict[data.current_dt.strftime('%Y-%m-%d')]

    except:
        return
    
    positions = {e.symbol: p.amount * p.last_sale_price
                 for e, p in context.portfolio.positions.items()}


    stock_to_sell = [k for k in positions if k not in ratio_today]
    
    for instrument in stock_to_sell:
        context.order_target( context.symbol(instrument), 0)
        
    for instrument in ratio_today:
        context.order_target_percent( context.symbol(instrument), ratio_today[instrument])
    
# 回测引擎：准备数据，只执行一次
def m19_prepare_bigquant_run(context):
    pass

# 回测引擎：初始化函数，只执行一次
def m14_initialize_bigquant_run(context):
    # 加载预测数据
    context.ranker_prediction = context.options['data'].read_df()

    # 系统已经设置了默认的交易手续费和滑点，要修改手续费可使用如下函数
    context.set_commission(PerOrder(buy_cost=0.0003, sell_cost=0.0013, min_cost=5))
    # 预测数据，通过options传入进来，使用 read_df 函数，加载到内存 (DataFrame)
    # 设置买入的股票数量，这里买入预测股票列表排名靠前的5只
    stock_count = 5
    # 每只的股票的权重，如下的权重分配会使得靠前的股票分配多一点的资金，[0.339160, 0.213986, 0.169580, ..]
    context.stock_weights = T.norm([1 / math.log(i + 2) for i in range(0, stock_count)])
    # 设置每只股票占用的最大资金比例
    context.max_cash_per_instrument = 0.2
    context.hold_days = 5

# 回测引擎：每日数据处理函数，每天执行一次
def m14_handle_data_bigquant_run(context, data):
    # 按日期过滤得到今日的预测数据
    ranker_prediction = context.ranker_prediction[
        context.ranker_prediction.date == data.current_dt.strftime('%Y-%m-%d')]

    # 1. 资金分配
    # 平均持仓时间是hold_days，每日都将买入股票，每日预期使用 1/hold_days 的资金
    # 实际操作中，会存在一定的买入误差，所以在前hold_days天，等量使用资金；之后，尽量使用剩余资金（这里设置最多用等量的1.5倍）
    is_staging = context.trading_day_index < context.hold_days # 是否在建仓期间（前 hold_days 天）
    cash_avg = context.portfolio.portfolio_value / context.hold_days
    cash_for_buy = min(context.portfolio.cash, (1 if is_staging else 1.5) * cash_avg)
    cash_for_sell = cash_avg - (context.portfolio.cash - cash_for_buy)
    positions = {e.symbol: p.amount * p.last_sale_price
                 for e, p in context.perf_tracker.position_tracker.positions.items()}

    # 2. 生成卖出订单：hold_days天之后才开始卖出；对持仓的股票，按StockRanker预测的排序末位淘汰
    if not is_staging and cash_for_sell > 0:
        equities = {e.symbol: e for e, p in context.perf_tracker.position_tracker.positions.items()}
        instruments = list(reversed(list(ranker_prediction.instrument[ranker_prediction.instrument.apply(
                lambda x: x in equities and not context.has_unfinished_sell_order(equities[x]))])))
        # print('rank order for sell %s' % instruments)
        for instrument in instruments:
            context.order_target(context.symbol(instrument), 0)
            cash_for_sell -= positions[instrument]
            if cash_for_sell <= 0:
                break

    # 3. 生成买入订单：按StockRanker预测的排序，买入前面的stock_count只股票
    buy_cash_weights = context.stock_weights
    buy_instruments = list(ranker_prediction.instrument[:len(buy_cash_weights)])
    max_cash_per_instrument = context.portfolio.portfolio_value * context.max_cash_per_instrument
    for i, instrument in enumerate(buy_instruments):
        cash = cash_for_buy * buy_cash_weights[i]
        if cash > max_cash_per_instrument - positions.get(instrument, 0):
            # 确保股票持仓量不会超过每次股票最大的占用资金量
            cash = max_cash_per_instrument - positions.get(instrument, 0)
        if cash > 0:
            context.order_value(context.symbol(instrument), cash)

# 回测引擎：准备数据，只执行一次
def m14_prepare_bigquant_run(context):
    pass

# 回测引擎：每个单位时间开始前调用一次，即每日开盘前调用一次。
def m14_before_trading_start_bigquant_run(context, data):
    pass


m1 = M.instruments.v2(
    start_date='2010-01-01',
    end_date='2015-01-01',
    market='CN_STOCK_A',
    instrument_list='',
    max_count=0
)

m2 = M.advanced_auto_labeler.v2(
    instruments=m1.data,
    label_expr="""# #号开始的表示注释
# 0. 每行一个，顺序执行，从第二个开始，可以使用label字段
# 1. 可用数据字段见 https://bigquant.com/docs/develop/datasource/deprecated/history_data.html
#   添加benchmark_前缀，可使用对应的benchmark数据
# 2. 可用操作符和函数见 `表达式引擎 <https://bigquant.com/docs/develop/bigexpr/usage.html>`_

# 计算收益：5日收盘价(作为卖出价格)除以明日开盘价(作为买入价格)
shift(close, -5) / shift(open, -1)

# 极值处理：用1%和99%分位的值做clip
clip(label, all_quantile(label, 0.01), all_quantile(label, 0.99))

# 将分数映射到分类，这里使用20个分类
all_wbins(label, 20)

# 过滤掉一字涨停的情况 (设置label为NaN，在后续处理和训练中会忽略NaN的label)
where(shift(high, -1) == shift(low, -1), NaN, label)
""",
    start_date='',
    end_date='',
    benchmark='000300.SHA',
    drop_na_label=True,
    cast_label_int=True
)

m3 = M.input_features.v1(
    features="""# #号开始的表示注释
# 多个特征，每行一个，可以包含基础特征和衍生特征
return_5
return_10
return_20
avg_amount_0/avg_amount_5
avg_amount_5/avg_amount_20
rank_avg_amount_0/rank_avg_amount_5
rank_avg_amount_5/rank_avg_amount_10
rank_return_0
rank_return_5
rank_return_10
rank_return_0/rank_return_5
rank_return_5/rank_return_10
pe_ttm_0
"""
)

m15 = M.general_feature_extractor.v7(
    instruments=m1.data,
    features=m3.data,
    start_date='',
    end_date='',
    before_start_days=90
)

m16 = M.derived_feature_extractor.v3(
    input_data=m15.data,
    features=m3.data,
    date_col='date',
    instrument_col='instrument',
    drop_na=False,
    remove_extra_columns=False
)

m7 = M.join.v3(
    data1=m2.data,
    data2=m16.data,
    on='date,instrument',
    how='inner',
    sort=False
)

m5 = M.dropnan.v2(
    input_data=m7.data
)

m4 = M.stock_ranker_train.v6(
    training_ds=m5.data,
    features=m3.data,
    learning_algorithm='排序',
    number_of_leaves=10,
    minimum_docs_per_leaf=1000,
    number_of_trees=20,
    learning_rate=0.1,
    max_bins=1023,
    feature_fraction=1,
    data_row_fraction=1,
    plot_charts=True,
    ndcg_discount_base=1,
    m_lazy_run=False
)

m12 = M.input_features.v1(
    features_ds=m3.data,
    features="""
# #号开始的表示注释，注释需单独一行
# 多个特征，每行一个，可以包含基础特征和衍生特征，特征须为本平台特征
list_days_0
"""
)

m9 = M.instruments.v2(
    start_date=T.live_run_param('trading_date', '2016-01-01'),
    end_date=T.live_run_param('trading_date', '2017-01-01'),
    market='CN_STOCK_A',
    instrument_list='',
    max_count=0
)

m17 = M.general_feature_extractor.v7(
    instruments=m9.data,
    features=m12.data,
    start_date='',
    end_date='',
    before_start_days=90
)

m18 = M.derived_feature_extractor.v3(
    input_data=m17.data,
    features=m12.data,
    date_col='date',
    instrument_col='instrument',
    drop_na=False,
    remove_extra_columns=False
)

m13 = M.filter.v3(
    input_data=m18.data,
    expr='list_days_0>200',
    output_left_data=False
)

m10 = M.dropnan.v2(
    input_data=m13.data
)

m8 = M.stock_ranker_predict.v5(
    model=m4.model,
    data=m10.data,
    m_lazy_run=False
)

m6 = M.cached.v3(
    input_1=m8.predictions,
    input_2=m9.data,
    run=m6_run_bigquant_run,
    post_run=m6_post_run_bigquant_run,
    input_ports='',
    params='{}',
    output_ports=''
)

m11 = M.classical_portfolio_optimizer.v6(
    input_1=m6.data_1,
    date='',
    weight_sum=1,
    upper_weight=0.6,
    lower_weight=0.01,
    before_start_days=10,
    target='最大化收益',
    return_equal_weight_if_fail=True
)

m19 = M.trade.v4(
    instruments=m9.data,
    options_data=m11.data_1,
    start_date='',
    end_date='',
    initialize=m19_initialize_bigquant_run,
    handle_data=m19_handle_data_bigquant_run,
    prepare=m19_prepare_bigquant_run,
    volume_limit=0.025,
    order_price_field_buy='open',
    order_price_field_sell='close',
    capital_base=1000000,
    auto_cancel_non_tradable_orders=True,
    data_frequency='daily',
    price_type='真实价格',
    product_type='股票',
    plot_charts=True,
    backtest_only=False,
    benchmark='000300.SHA'
)

m14 = M.trade.v4(
    instruments=m9.data,
    options_data=m8.predictions,
    start_date='',
    end_date='',
    initialize=m14_initialize_bigquant_run,
    handle_data=m14_handle_data_bigquant_run,
    prepare=m14_prepare_bigquant_run,
    before_trading_start=m14_before_trading_start_bigquant_run,
    volume_limit=0.025,
    order_price_field_buy='open',
    order_price_field_sell='close',
    capital_base=1000000,
    auto_cancel_non_tradable_orders=True,
    data_frequency='daily',
    price_type='真实价格',
    product_type='股票',
    plot_charts=True,
    backtest_only=False,
    benchmark=''
)


[2021-12-14 08:14:56.143654] INFO: moduleinvoker: instruments.v2 开始运行..

[2021-12-14 08:14:56.161550] INFO: moduleinvoker: 命中缓存

[2021-12-14 08:14:56.162948] INFO: moduleinvoker: instruments.v2 运行完成[0.01931s].

[2021-12-14 08:14:56.177508] INFO: moduleinvoker: advanced_auto_labeler.v2 开始运行..

[2021-12-14 08:14:56.186837] INFO: moduleinvoker: 命中缓存

...

[2021-12-14 08:21:25.048491] INFO: Performance: Simulated 244 trading days out of 244.

[2021-12-14 08:21:25.049935] INFO: Performance: first open: 2016-01-04 09:30:00+00:00

[2021-12-14 08:21:25.051054] INFO: Performance: last close: 2016-12-30 15:00:00+00:00

[2021-12-14 08:21:27.734622] INFO: moduleinvoker: backtest.v8 运行完成[11.747051s].

[2021-12-14 08:21:27.736010] INFO: moduleinvoker: trade.v4 运行完成[13.399075s].